In [0]:
%sh ls -l

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
dbutils.widgets.text("config_file", "dev-config.json")

In [0]:
config_file = dbutils.widgets.get("config_file")

In [0]:
import json

with open(config_file) as file:
    config = json.load(file)
print(config)

In [0]:
import pandas as pd

if config["env"] == "dev":
  df = pd.read_csv("sample-data.csv")
elif config["env"] == "prod":
  df = pd.read_csv("/Volumes/path/to/data")
else:
  raise Exception("Invalid environment")

In [0]:
from mylib import sumRows
print(sumRows(df))